# create masked version of tiffs

In [1]:
import skimage
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile
import warnings
warnings.filterwarnings('ignore')

We do manual region masking for host and fecal regions in FIJI, and generate masks for fecal regions

these fecal region masks were saved and used in this script

In [2]:
# fecal side channels
f_side_only = ["Alx488.tiff",
         "DIG.tiff",
         "FITC.tiff",
         "TAMRA.tiff"]

# channels can be on both side
both_side = ["dsDNA.tiff",
            "IgA.tiff",
            "MUC2.tiff",
            "Reg3beta.tiff"]

# rest should be only in host side

extracted_imgs_dir = '../../final_data/compensated/micro_fine1ms/' # all extracted data stored here
out_img_dir = '../../final_data/masked/micro_fine1ms/'

#### runs to create tissue seg
#run_names = ['2022-12-11T15-07-08-C3-3pt-fine-2ms-1024-400',
#             '2022-12-10T18-30-32-T3D5-5pt-Fine-2ms-5ul-Test'] # specific run name
#run_names = ['2023-02-28T12-53-51-DSST1-Run1'] # specific run name
#run_names = ['2023-03-04T16-22-15-DSSC1-Run1']
#run_names = ['2023-03-04T16-22-15-DSSC1-Run1', '2023-03-04T19-46-22-DSSC1-Run2',
#            '2023-03-04T23-36-59-DSSC1-Run3', '2023-03-05T15-51-42-DSST2-Run1',
#            '2023-03-05T18-22-42-DSST2-Run2', '2023-03-05T21-42-35-DSST2-Run3',
#            '2023-03-05T23-58-06-DSST2-Run4', '2023-03-06T11-23-28-DSST2-Run5',
#            '2023-03-06T13-40-46-DSST2-Run6', '2023-03-06T16-15-43-DSSC2-Run1'] # specific run name

#run_names = ['2023-03-04T16-22-15-DSSC1-Run1', '2023-03-04T19-46-22-DSSC1-Run2',
#             '2023-03-04T23-36-59-DSSC1-Run3', '2023-03-05T15-51-42-DSST2-Run1',
#             '2023-03-05T18-22-42-DSST2-Run2', '2023-03-05T21-42-35-DSST2-Run3',
#             '2023-03-05T23-58-06-DSST2-Run4', '2023-03-06T11-23-28-DSST2-Run5',
#             '2023-03-06T13-40-46-DSST2-Run6', '2023-03-06T16-15-43-DSSC2-Run1',
#             '2023-03-07T18-32-49-DSSC2-Run5', '2023-03-07T16-08-39-DSSC2-Run4',
#             '2023-03-07T13-54-48-DSSC2-Run3', '2023-03-07T11-17-56-DSSC2-Run2',
#             '2023-03-09T15-53-36-DSSC3-Run4', '2023-03-09T13-34-52-DSSC3-Run3',
#             '2023-03-09T10-32-36-DSSC3-Run2', '2023-03-08T23-51-06-DSSC3-Run1',
#             '2023-03-08T21-26-09-DSST3-Run6', '2023-03-08T18-48-43-DSST3-Run5',
#             '2023-03-08T16-15-19-DSST3-Run4', '2023-03-08T14-03-50-DSST3-Run3',
#             '2023-03-08T11-39-28-DSST3-Run2', '2023-03-07T22-15-40-DSST3-Run1'] # specific run name

#run_names = ['2023-03-08T11-39-28-DSST3-Run2'] # rerun this due to missing

run_names = ['2023-03-10T12-31-44-DSST4-Run1', '2023-03-10T14-47-38-DSST4-Run2', '2023-03-10T17-34-14-DSST4-Run3',
            '2023-03-18T14-36-10-DSST4-Run4', '2023-03-18T17-28-03-DSST4-Run5', '2023-03-18T23-11-36-DSST4-Run6',
            '2023-03-19T12-45-44-DSSC4-Run1', '2023-03-19T15-20-40-DSSC4-Run2', '2023-03-19T17-38-15-DSSC4-Run3',
            '2023-03-19T20-32-23-DSSC4-Run4', '2023-03-20T12-07-46-DSST1-Run1', '2023-03-20T14-32-22-DSST1-Run2',
            '2023-03-20T19-13-53-DSST1-Run3', '2023-03-20T21-44-51-DSST1-Run4', '2023-03-21T24-29-43-DSST1-Run5',
            '2023-03-21T10-32-42-DSST1-Run6', '2023-03-21T12-44-51-DSST1-Run7', '2023-03-28T11-45-15-DSSC1-Run4'
            ]
####

runs = run_names

In [3]:
for run in runs:
    path_compensated = os.path.join(extracted_imgs_dir, run)
    fovs = os.listdir(path_compensated)
    
    for fov in fovs:
        if fov == '.DS_Store':
            continue
        
        # read in mask if exist
        mask_dir = os.path.join(extracted_imgs_dir, run, fov,"rescaled", "Mask.tif")
        
        #### start the process
        fov_dir = os.path.join(extracted_imgs_dir, run, fov, "rescaled")
        out_dir = os.path.join(out_img_dir, run, fov)
        channels = os.listdir(fov_dir)
        channels.remove("tissue_add.tiff")
        
        os.makedirs(out_dir,exist_ok=True)
        
        if os.path.exists(mask_dir) != True: # case where no fecal side imaged
            print("no fecal side in this fov")
            print(run + fov)
            for channel in channels:
                if channel not in f_side_only:
                    if channel == "dsDNA.tiff":
                        img = skimage.io.imread(os.path.join(fov_dir,channel))
                        skimage.io.imsave(os.path.join(out_dir, "dsDNA_host.tiff"),img)
                    else:
                        img = skimage.io.imread(os.path.join(fov_dir,channel))
                        skimage.io.imsave(os.path.join(out_dir, channel),img)
                else: # if dont have feces region just save out empty img
                    skimage.io.imsave(os.path.join(out_dir, channel),np.zeros((1024,1024)))
                    
        else:
            mask = skimage.io.imread(mask_dir)
            mask[mask > 0] = 1 # make mask to 01 instead of 0255
            
            for channel in channels:
                
                if channel in f_side_only: # fecal side only channels
                    img = skimage.io.imread(os.path.join(fov_dir,channel))
                    img2 = img * (1 - mask)
                    skimage.io.imsave(os.path.join(out_dir, channel),img2)
                    
                elif ((channel not in f_side_only) and (channel not in both_side)): # host side only channels
                    img = skimage.io.imread(os.path.join(fov_dir,channel))
                    img2 = img * mask
                    skimage.io.imsave(os.path.join(out_dir, channel),img2)
                    
                else: # ones that can be on both side
                    if channel == "dsDNA.tiff":
                        img = skimage.io.imread(os.path.join(fov_dir,channel))
                        img2 = img * mask
                        skimage.io.imsave(os.path.join(out_dir, "dsDNA_host.tiff"),img2)
                        img2 = img * (1 - mask)
                        skimage.io.imsave(os.path.join(out_dir, "dsDNA_feces.tiff"),img2)
                        
                    else:
                        img = skimage.io.imread(os.path.join(fov_dir,channel))
                        skimage.io.imsave(os.path.join(out_dir, channel),img)
                    
                    
            

no fecal side in this fov
2023-03-19T15-20-40-DSSC4-Run2fov-3-scan-1
